# Commercial Bank Customer Retention Prediction

## APSTA-GE.2401: Statistical Consulting

## Scripts

Created on: 12/04/2020

Modified on: 12/06/2020

## Data Preprocess

### Description

This script preprocesses the raw data originated from the competition data warehouse. There are three main data packages:

- `x_train`: the train data package containing all features.
- `y_train`: the data package for feature test.
- `x_test`: the train data package for prediction. It contains the same features as `x_train`.

We started from examining the `y_train` because it contains labels that can validate our model predictions. `y_train` contains random sampled label data from two quarters: Q3_2020 and Q4_2020. Since the customer ID column, `cust_no`, only contains unique values, we determined our data processing strategies as follows:

1. Use **quarter** to separate data processing procedures. We created two training sets, `X_train_Q3` and `X_train_Q4`, and merged them before applying models. In this way, we bypassed duplicated customer IDs in both `y_train` sets caused by random sampling. This allowed us to maximize the number of labels that can be validated.

    - `y_Q3_3` contains 69126 rows, `y_Q3_3` contains 76170 rows.
    - `y_train` has 62397 duplicated customer IDs. 
    - `y_train` has 40090 completely identical records (same customer ID, same label).
    - Two samples are heavily overlapped.
    - 22307 customers changed their churn preference from Q3 to Q4.
    
2. Based on quarterly-separated `y_train` set, we merged `X_train` raw data accordingly. For each quarter, we dropped duplicated customer IDs except for the last occurrance.

3. During data preprocessing, we examined records in the `cust_avli` column of the `X_train` sets. These sets contain the ID of all effective customers. We confirmed that these ID are the same as those in the `y_train` set. Therefore, we trimmed the dataset based on the `cust_no` column in the `cust_avli`, separated by quarters.

    - Confirmed that `cust_avli` is the key indexing column.

In [2]:
import pandas as pd
import numpy as np
import csv
import glob
import re
import os

print('SUCCESS! All modules are imported.')

SUCCESS! All modules are imported.


In [157]:
def merge_file(path):
    '''Concatenate files from path
    Param: path: file path from which data are imported and concatenated
    '''
    file_names = [name for name in glob.glob(path)]
    df_temp = (pd.read_csv(file) for file in file_names)
    return(pd.concat(df_temp, ignore_index=True, axis='index'))

----

## y_train

In this step, we proved that there were duplicated customer IDs in each `y_train` sets. We did not perform trimming because even for the same customer, it is possible that he/she changed his/her churn preference after a period of time.

In [261]:
path = '../data/raw/y_train_3/y_Q[34]_3.csv'
y_train = merge_file(path)

In [476]:
y_train.to_csv('../data/preprocess/y_train.csv', index=False)

In [280]:
path = '../data/raw/y_train_3/'
y_Q3_3 = pd.read_csv(path + 'y_Q3_3.csv')
y_Q4_3 = pd.read_csv(path + 'y_Q4_3.csv')

In [281]:
# Index for trimming
idxQ3 = y_Q3_3['cust_no']
idxQ4 = y_Q4_3['cust_no']

In [294]:
print(len(idxQ3))
print(len(idxQ4))

69126
76170


In [287]:
def trim_by_quarter(dat, isQ3):
    '''Trim the data by quarterly index
    Param: dat: imported data
    Param: isQ3: binary, 1 if Q3; 0 if else
    '''
    if isQ3:
        return(dat[dat['cust_no'].isin(idxQ3)])
    else:
        return(dat[dat['cust_no'].isin(idxQ4)])

In [265]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
# Confirm that all customer IDs are unique
print(len(y_Q3_3['cust_no'].value_counts()) == y_Q3_3.shape[0])
print(len(y_Q4_3['cust_no'].value_counts()) == y_Q4_3.shape[0])

True
True


In [266]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('y_Q3_3 has {} rows and {} columns.'.format(y_Q3_3.shape[0], y_Q3_3.shape[1]))
print('y_Q4_3 has {} rows and {} columns.'.format(y_Q4_3.shape[0], y_Q4_3.shape[1]))
print('y_train has {} rows and {} columns.'.format(y_train.shape[0], y_train.shape[1]))
print('y_train has {} duplicated customer IDs.'.format(y_train['cust_no'].duplicated().sum()))
print('y_train has {} completely identical records.'.format(y_train.duplicated().sum()))
print('{} customers changed their churn preference from Q3 to Q4.'.format(
    y_train['cust_no'].duplicated().sum()-y_train.duplicated().sum()))

y_Q3_3 has 69126 rows and 2 columns.
y_Q4_3 has 76170 rows and 2 columns.
y_train has 145296 rows and 2 columns.
y_train has 62397 duplicated customer IDs.
y_train has 40090 completely identical records.
22307 customers changed their churn preference from Q3 to Q4.


----

### Sample Submission File

In [244]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
sample = pd.read_csv('../instructions/sample_submission.csv')
print('Sample submission file has {} rows and {} columns.'.format(sample.shape[0], sample.shape[1]))
display(sample.head())

Sample submission file has 76722 rows and 2 columns.


,cust_no,label
0,0x3b9b4615,0
1,0x3b9ae61b,0
2,0x3b9add69,0
3,0x3b9b3601,0
4,0x3b9b2599,0


----

## X_train

In [461]:
X_train_Q3 = y_Q3_3.drop('label', axis=1).copy()
X_train_Q4 = y_Q4_3.drop('label', axis=1).copy()

### Customer Assets (aum)

In [377]:
# Q3
path = '../data/raw/x_train/aum_m[789].csv'
aum_Q3 = merge_file(path)

In [378]:
# Q4
path = '../data/raw/x_train/aum_m1[012].csv'
aum_Q4 = merge_file(path)

In [379]:
# Drop duplicated customer IDs except for the last occurrance
aum_Q3 = aum_Q3.drop_duplicates(subset=['cust_no'], keep='last')
aum_Q4 = aum_Q4.drop_duplicates(subset=['cust_no'], keep='last')

In [380]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, aum_Q3 has {} rows and {} columns.'.format(aum_Q3.shape[0], aum_Q3.shape[1]))
print('After dropping duplicated customer IDs, aum_Q4 has {} rows and {} columns.'.format(aum_Q4.shape[0], aum_Q4.shape[1]))

After dropping duplicated customer IDs, aum_Q3 has 493441 rows and 9 columns.
After dropping duplicated customer IDs, aum_Q4 has 543823 rows and 9 columns.


In [381]:
# Trim by the customer IDs in `y_train` set, separated by quarter
aum_Q3 = trim_by_quarter(aum_Q3, True)
aum_Q4 = trim_by_quarter(aum_Q4, False)

In [382]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After trimming, aum_Q3 has {} rows and {} columns.'.format(aum_Q3.shape[0], aum_Q3.shape[1]))
print('After trimming, aum_Q4 has {} rows and {} columns.'.format(aum_Q4.shape[0], aum_Q4.shape[1]))

After trimming, aum_Q3 has 69126 rows and 9 columns.
After trimming, aum_Q4 has 76170 rows and 9 columns.


In [386]:
# Save to archive
aum_Q3.to_csv('../data/preprocess/archive/aum_Q3.csv', index=False)
aum_Q4.to_csv('../data/preprocess/archive/aum_Q4.csv', index=False)

In [462]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(aum_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(aum_Q4, how='left', on='cust_no')

### Customer Behavior (behavior)

In [184]:
# Q3
path = '../data/raw/x_train/behavior_m[789].csv'
behavior_Q3 = merge_file(path)

In [185]:
# Q4
path = '../data/raw/x_train/behavior_m1[012].csv'
behavior_Q4 = merge_file(path)

In [186]:
# Drop duplicated customer IDs except for the last occurrance
behavior_Q3 = behavior_Q3.drop_duplicates(subset=['cust_no'], keep='last')
behavior_Q4 = behavior_Q4.drop_duplicates(subset=['cust_no'], keep='last')

In [187]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, behavior_Q3 has {} rows and {} columns.'.format(behavior_Q3.shape[0], behavior_Q3.shape[1]))
print('After dropping duplicated customer IDs, behavior_Q4 has {} rows and {} columns.'.format(behavior_Q4.shape[0], behavior_Q4.shape[1]))

After dropping duplicated customer IDs, behavior_Q3 has 493441 rows and 8 columns.
After dropping duplicated customer IDs, behavior_Q4 has 543823 rows and 8 columns.


In [389]:
# Trim by the customer IDs in `y_train` set, separated by quarter
behavior_Q3 = trim_by_quarter(behavior_Q3, True)
behavior_Q4 = trim_by_quarter(behavior_Q4, False)

In [390]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After trimming, behavior_Q3 has {} rows and {} columns.'.format(behavior_Q3.shape[0], behavior_Q3.shape[1]))
print('After trimming, behavior_Q4 has {} rows and {} columns.'.format(behavior_Q4.shape[0], behavior_Q4.shape[1]))

After trimming, behavior_Q3 has 69126 rows and 8 columns.
After trimming, behavior_Q4 has 76170 rows and 8 columns.


In [391]:
# Save to archive
behavior_Q3.to_csv('../data/preprocess/archive/behavior_Q3.csv', index=False)
behavior_Q4.to_csv('../data/preprocess/archive/behavior_Q4.csv', index=False)

In [463]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(behavior_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(behavior_Q4, how='left', on='cust_no')

### Important Customer Behavior (big_event)

In [189]:
# Q3
path = '../data/raw/x_train/big_event_Q3.csv'
big_event_Q3 = merge_file(path)

<ipython-input-157-cee1144e6e7f>:7: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  return(pd.concat(df_temp, ignore_index=True, axis='index'))


In [190]:
# Q4
path = '../data/raw/x_train/big_event_Q4.csv'
big_event_Q4 = merge_file(path)

In [193]:
# Drop duplicated customer IDs except for the last occurrance
big_event_Q3 = big_event_Q3.drop_duplicates(subset=['cust_no'], keep='last')
big_event_Q3 = big_event_Q4.drop_duplicates(subset=['cust_no'], keep='last')

In [194]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, big_event_Q3 has {} rows and {} columns.'.format(big_event_Q3.shape[0], big_event_Q3.shape[1]))
print('After dropping duplicated customer IDs, big_event_Q4 has {} rows and {} columns.'.format(big_event_Q4.shape[0], big_event_Q4.shape[1]))

After dropping duplicated customer IDs, big_event_Q3 has 493441 rows and 19 columns.
After dropping duplicated customer IDs, big_event_Q4 has 543823 rows and 19 columns.


In [393]:
# Trim by the customer IDs in `y_train` set, separated by quarter
big_event_Q3 = trim_by_quarter(big_event_Q3, True)
big_event_Q4 = trim_by_quarter(big_event_Q4, False)

In [394]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After trimming, big_event_Q3 has {} rows and {} columns.'.format(big_event_Q3.shape[0], big_event_Q3.shape[1]))
print('After trimming, big_event_Q4 has {} rows and {} columns.'.format(big_event_Q4.shape[0], big_event_Q4.shape[1]))

After trimming, big_event_Q3 has 69126 rows and 19 columns.
After trimming, big_event_Q4 has 76170 rows and 19 columns.


In [395]:
# Save to archive
big_event_Q3.to_csv('../data/preprocess/archive/big_event_Q3.csv', index=False)
big_event_Q4.to_csv('../data/preprocess/archive/big_event_Q4.csv', index=False)

In [464]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(big_event_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(big_event_Q4, how='left', on='cust_no')

### Customer Deposits (cunkuan)

In [398]:
# Q3
path = '../data/raw/x_train/cunkuan_m[789].csv'
savings_Q3 = merge_file(path)

In [399]:
# Q4
path = '../data/raw/x_train/cunkuan_m1[012].csv'
savings_Q4 = merge_file(path)

In [400]:
# Drop duplicated customer IDs except for the last occurrance
savings_Q3 = savings_Q3.drop_duplicates(subset=['cust_no'], keep='last')
savings_Q4 = savings_Q4.drop_duplicates(subset=['cust_no'], keep='last')

In [401]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, savings_Q3 has {} rows and {} columns.'.format(savings_Q3.shape[0], savings_Q3.shape[1]))
print('After dropping duplicated customer IDs, savings_Q4 has {} rows and {} columns.'.format(savings_Q4.shape[0], savings_Q4.shape[1]))

After dropping duplicated customer IDs, savings_Q3 has 200721 rows and 3 columns.
After dropping duplicated customer IDs, savings_Q4 has 237049 rows and 3 columns.


In [402]:
# Trim by the customer IDs in `y_train` set, separated by quarter
savings_Q3 = trim_by_quarter(savings_Q3, True)
savings_Q4 = trim_by_quarter(savings_Q4, False)

In [403]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After trimming, savings_Q3 has {} rows and {} columns.'.format(savings_Q3.shape[0], savings_Q3.shape[1]))
print('After trimming, savings_Q4 has {} rows and {} columns.'.format(savings_Q4.shape[0], savings_Q4.shape[1]))

After trimming, savings_Q3 has 69122 rows and 3 columns.
After trimming, savings_Q4 has 76167 rows and 3 columns.


In [404]:
# Save to archive
savings_Q3.to_csv('../data/preprocess/archive/savings_Q3.csv', index=False)
savings_Q4.to_csv('../data/preprocess/archive/savings_Q4.csv', index=False)

In [465]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(savings_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(savings_Q4, how='left', on='cust_no')

### Valid Customer (cust_avli)

Valid customer set contains customer IDs that are same as those in the test set.

In [204]:
# Q3
path = '../data/raw/x_train/cust_avli_Q3.csv'
cust_avli_Q3 = merge_file(path)

In [205]:
# Q4
path = '../data/raw/x_train/cust_avli_Q4.csv'
cust_avli_Q4 = merge_file(path)

In [212]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('cust_avli_Q3 has {} rows and {} columns.'.format(cust_avli_Q3.shape[0], cust_avli_Q3.shape[1]))
print('cust_avli_Q4 has {} rows and {} columns.'.format(cust_avli_Q4.shape[0], cust_avli_Q4.shape[1]))

cust_avli_Q3 has 69126 rows and 1 columns.
cust_avli_Q4 has 76170 rows and 1 columns.


In [211]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
# Confirm that valid customers are the same as those in the test set
print(len(cust_avli_Q3.value_counts()) == y_Q3_3.shape[0])
print(len(cust_avli_Q4.value_counts()) == y_Q4_3.shape[0])

True
True


### Customer Trivias (cust_info)

In [451]:
# Q3
path = '../data/raw/x_train/cust_info_q3.csv'
cust_info_Q3 = merge_file(path)

In [452]:
# Q4
path = '../data/raw/x_train/cust_info_q4.csv'
cust_info_Q4 = merge_file(path)

In [453]:
# Drop duplicated customer IDs except for the last occurrance
cust_info_Q3 = cust_info_Q3.drop_duplicates(subset=['cust_no'], keep='last')
cust_info_Q4 = cust_info_Q4.drop_duplicates(subset=['cust_no'], keep='last')

In [454]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, cust_info_Q3 has {} rows and {} columns.'.format(cust_info_Q3.shape[0], cust_info_Q3.shape[1]))
print('After dropping duplicated customer IDs, cust_info_Q4 has {} rows and {} columns.'.format(cust_info_Q4.shape[0], cust_info_Q4.shape[1]))

After dropping duplicated customer IDs, cust_info_Q3 has 493441 rows and 21 columns.
After dropping duplicated customer IDs, cust_info_Q4 has 543823 rows and 21 columns.


In [455]:
# Trim by the customer IDs in `y_train` set, separated by quarter
cust_info_Q3 = trim_by_quarter(cust_info_Q3, True)
cust_info_Q4 = trim_by_quarter(cust_info_Q4, False)

In [456]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After trimming, cust_info_Q3 has {} rows and {} columns.'.format(cust_info_Q3.shape[0], cust_info_Q3.shape[1]))
print('After trimming, cust_info_Q4 has {} rows and {} columns.'.format(cust_info_Q4.shape[0], cust_info_Q4.shape[1]))

After trimming, cust_info_Q3 has 69126 rows and 21 columns.
After trimming, cust_info_Q4 has 76170 rows and 21 columns.


In [457]:
# Save to archive
cust_info_Q3.to_csv('../data/preprocess/archive/cust_info_Q3.csv', index=False)
cust_info_Q4.to_csv('../data/preprocess/archive/cust_info_Q4.csv', index=False)

In [470]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(cust_info_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(cust_info_Q4, how='left', on='cust_no')

### X_Train Ready

In [471]:
print(X_train_Q3.shape)
print(X_train_Q4.shape)

(69126, 56)
(76170, 56)


In [472]:
X_train_Q3.to_csv('../data/preprocess/archive/X_train_Q3.csv', index=False)
X_train_Q4.to_csv('../data/preprocess/archive/X_train_Q4.csv', index=False)

In [473]:
X_train = X_train_Q3.append(X_train_Q4)

In [474]:
X_train.shape

(145296, 56)

In [475]:
X_train.to_csv('../data/preprocess/X_train.csv', index=False)

----

## X_test

### Customer Assets (aum)

In [220]:
# Q1
path = '../data/raw/x_test/aum_m[123].csv'
aum_Q1 = merge_file(path)

In [221]:
# Drop duplicated customer IDs except for the last occurrance
aum_Q1 = aum_Q1.drop_duplicates(subset=['cust_no'], keep='last')

In [222]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, aum_Q1 has {} rows and {} columns.'.format(aum_Q1.shape[0], aum_Q1.shape[1]))

After dropping duplicated customer IDs, aum_Q1 has 659624 rows and 9 columns.


### Customer Behavior (behavior)

In [246]:
# Q1
path = '../data/raw/x_test/behavior_m[123].csv'
behavior_Q1 = merge_file(path)

In [247]:
# Drop duplicated customer IDs except for the last occurrance
behavior_Q1 = behavior_Q1.drop_duplicates(subset=['cust_no'], keep='last')

In [248]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, behavior_Q1 has {} rows and {} columns.'.format(behavior_Q1.shape[0], behavior_Q1.shape[1]))

After dropping duplicated customer IDs, behavior_Q1 has 659624 rows and 8 columns.


### Important Customer Behavior (big_event)

In [249]:
# Q1
path = '../data/raw/x_test/big_event_Q1.csv'
big_event_Q1 = merge_file(path)

In [250]:
# Drop duplicated customer IDs except for the last occurrance
big_event_Q1 = big_event_Q1.drop_duplicates(subset=['cust_no'], keep='last')

In [251]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, big_event_Q1 has {} rows and {} columns.'.format(big_event_Q1.shape[0], big_event_Q1.shape[1]))

After dropping duplicated customer IDs, big_event_Q1 has 659624 rows and 19 columns.


### Customer Deposits (cunkuan)

In [252]:
# Q1
path = '../data/raw/x_test/cunkuan_m[123].csv'
savings_Q1 = merge_file(path)

In [253]:
# Drop duplicated customer IDs except for the last occurrance
savings_Q1 = savings_Q1.drop_duplicates(subset=['cust_no'], keep='last')

In [254]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, savings_Q1 has {} rows and {} columns.'.format(savings_Q1.shape[0], savings_Q1.shape[1]))

After dropping duplicated customer IDs, savings_Q1 has 254816 rows and 3 columns.


### Valid Customer (cust_avli)

The `cust_avli` in the `x_test` sets is identical to the sample submission file.

In [255]:
# Q1
path = '../data/raw/x_test/cust_avli_Q1.csv'
cust_avli_Q1 = merge_file(path)

In [256]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('cust_avli_Q1 has {} rows and {} columns.'.format(cust_avli_Q1.shape[0], cust_avli_Q1.shape[1]))

cust_avli_Q1 has 76722 rows and 1 columns.


### Customer Trivias (cust_info)

In [257]:
# Q1
path = '../data/raw/x_test/cust_info_q1.csv'
cust_info_Q1 = merge_file(path)

In [ ]:
# Drop duplicated customer IDs except for the last occurrance
cust_info_Q1 = cust_info_Q1.drop_duplicates(subset=['cust_no'], keep='last')

In [258]:
# FOR DISPLAY PURPOSE. SKIP THIS CHUNK.
print('After dropping duplicated customer IDs, cust_info_Q1 has {} rows and {} columns.'.format(cust_info_Q1.shape[0], cust_info_Q1.shape[1]))

After dropping duplicated customer IDs, cust_info_Q1 has 659624 rows and 21 columns.


### X_test Ready

In [418]:
X_test = cust_avli_Q1.copy()

In [420]:
X_test = X_test.merge(aum_Q1, how='left', on='cust_no')

In [421]:
X_test = X_test.merge(behavior_Q1, how='left', on='cust_no')

In [422]:
X_test = X_test.merge(big_event_Q1, how='left', on='cust_no')

In [423]:
X_test = X_test.merge(savings_Q1, how='left', on='cust_no')

In [424]:
X_test = X_test.merge(cust_info_Q1, how='left', on='cust_no')

In [425]:
X_test.shape

(76722, 56)

In [477]:
X_test.to_csv('../data/preprocess/X_test.csv', index=False)

----

### Missing Values

In [139]:
# check which data set has missing values
def nulltracker(self):
    counter = 0
    for names in self:
        indicator = names.isnull().sum() == 0
        if indicator.all() == False:
            print(counter)
        counter = counter + 1 

In [127]:
aum_m = aum_m1, aum_m2, aum_m3, aum_m7, aum_m8, aum_m9, aum_m10, aum_m11, aum_m12
nulltracker(aum_m)

In [151]:
behavior_m = behavior_m1, behavior_m2, behavior_m3, behavior_m7, behavior_m8, behavior_m9, behavior_m10, behavior_m11, behavior_m12 
nulltracker(behavior_m)

2
5
8


In [129]:
cunkuan_m = cunkuan_m1, cunkuan_m2, cunkuan_m3, cunkuan_m7, cunkuan_m8, cunkuan_m9, cunkuan_m10, cunkuan_m11, cunkuan_m12
nulltracker(cunkuan_m)

In [141]:
big_event_Q = big_event_Q1, big_event_Q3, big_event_Q4
nulltracker(big_event_Q)

0
1
2


In [142]:
cust_avli_Q = cust_avli_Q1, cust_avli_Q3, cust_avli_Q4
nulltracker(cust_avli_Q)

In [143]:
cust_info_q = cust_info_q1, cust_info_q3, cust_info_q4
nulltracker(cust_info_q)

0
1
2


In [144]:
y_Q = y_Q3_3, y_Q4_3
nulltracker(y_Q)

In [146]:
#print(behavior_m[3][behavior_m[3].isnull().T.any()])
behavior_m[2].isnull().any()
behavior_m[2]["B6"]

0                         NaN
1                         NaN
2                         NaN
3         2020-03-31 22:06:00
4                         NaN
5         2020-03-30 19:07:00
6                         NaN
7                         NaN
8                         NaN
9                         NaN
10                        NaN
11                        NaN
12                        NaN
13                        NaN
14                        NaN
15        2020-01-16 11:07:00
16                        NaN
17                        NaN
18                        NaN
19                        NaN
20                        NaN
21                        NaN
22                        NaN
23                        NaN
24        2020-01-20 04:17:00
25                        NaN
26                        NaN
27                        NaN
28                        NaN
29                        NaN
                 ...         
659594                    NaN
659595                    NaN
659596    

In [156]:
behavior_m12

,cust_no,B1,B2,B3,B4,B5,B6,B7
0,0xb2d14994,5,2,1346.15,2,5346.15,2019-12-13 18:03:00,22
1,0xb2d65824,0,0,0.00,0,0.00,NaN,0
2,0xb2d539b7,0,0,0.00,0,0.00,NaN,0
3,0xb2d807ae,0,0,0.00,0,0.00,NaN,0
4,0xb2d176b2,14,3,292654.81,8,323939.53,2019-12-31 06:02:00,28
5,0xb2d1386f,0,0,0.00,0,0.00,NaN,0
6,0xb2d5ae1e,0,1,0.01,0,0.00,2019-12-13 19:35:00,1
7,0xb2d73522,0,0,0.00,0,0.00,NaN,0
8,0xb2d4bec7,0,0,0.00,0,0.00,NaN,0
9,0xb2d86da5,0,0,0.00,0,0.00,NaN,0


In [36]:
display(aum_m1.describe(), aum_m2.describe(), aum_m3.describe(), aum_m7.describe(), aum_m8.describe(), 
        aum_m9.describe(), aum_m10.describe(), aum_m11.describe(), aum_m12.describe())

,X1,X2,X3,X4,X5,X6,X7,X8
count,5.561950e+05,5.561950e+05,5.561950e+05,5.561950e+05,5.561950e+05,5.561950e+05,5.561950e+05,5.561950e+05
mean,4.752963e+04,3.296358e+03,4.633316e+03,4.351619e+03,1.939474e+02,3.158859e+02,6.446212e+04,1.264714e+04
std,1.555244e+06,5.476658e+05,2.170638e+05,1.000142e+05,1.039317e+04,2.743851e+04,6.525137e+05,1.866075e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,7.670000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.900000e+08,3.278498e+08,1.090412e+08,3.000000e+07,3.740000e+06,1.000000e+07,3.000000e+07,5.800000e+07


,X1,X2,X3,X4,X5,X6,X7,X8
count,6.036310e+05,6.036310e+05,6.036310e+05,6.036310e+05,6.036310e+05,6.036310e+05,6.036310e+05,6.036310e+05
mean,4.385846e+04,3.259812e+03,4.922958e+03,3.361816e+03,1.850724e+02,2.975686e+02,5.981933e+04,1.230564e+04
std,1.492338e+06,5.552339e+05,2.108768e+05,7.890369e+04,8.912695e+03,2.673754e+04,6.298647e+05,1.846377e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.770000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,3.970000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.900000e+08,3.335142e+08,1.090412e+08,1.877200e+07,2.100000e+06,1.000000e+07,3.000000e+07,5.800000e+07


,X1,X2,X3,X4,X5,X6,X7,X8
count,6.596240e+05,6.596240e+05,6.596240e+05,6.596240e+05,6.596240e+05,6.596240e+05,6.596240e+05,6.596240e+05
mean,4.102785e+04,2.397763e+03,4.548653e+03,3.144846e+03,1.842109e+02,2.599848e+02,5.723952e+04,1.273369e+04
std,1.397215e+06,4.778389e+05,2.230232e+05,7.184628e+04,1.528895e+04,2.447511e+04,6.255421e+05,3.088467e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.770000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,2.370000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.000000e+08,3.372508e+08,1.091459e+08,1.822000e+07,1.000000e+07,1.000000e+07,3.000000e+07,1.500000e+08


,X1,X2,X3,X4,X5,X6,X7,X8
count,4.654410e+05,4.654410e+05,4.654410e+05,4.654410e+05,4.654410e+05,4.654410e+05,4.654410e+05,4.654410e+05
mean,4.577886e+04,3.865932e+03,6.319957e+03,3.782432e+03,2.138875e+02,5.058526e+02,5.795086e+04,7.914900e+03
std,1.511018e+06,5.353912e+05,2.793072e+05,1.667316e+05,1.273293e+04,6.875766e+04,6.190032e+05,1.453283e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,4.300000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,3.470000e+08,2.106535e+08,1.000718e+08,6.800000e+07,4.035640e+06,4.075714e+07,3.475000e+07,5.000000e+07


,X1,X2,X3,X4,X5,X6,X7,X8
count,4.790630e+05,4.790630e+05,4.790630e+05,4.790630e+05,4.790630e+05,4.790630e+05,4.790630e+05,4.790630e+05
mean,5.015033e+04,3.243406e+03,4.790877e+03,3.527987e+03,2.071597e+02,4.809048e+02,5.968186e+04,8.685593e+03
std,1.633422e+06,5.083591e+05,2.026442e+05,1.213563e+05,1.190652e+04,6.803267e+04,6.276697e+05,1.541113e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,4.310000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.900000e+08,2.168897e+08,9.467516e+07,5.086000e+07,3.024484e+06,4.086245e+07,3.475000e+07,5.000000e+07


,X1,X2,X3,X4,X5,X6,X7,X8
count,4.934410e+05,4.934410e+05,4.934410e+05,4.934410e+05,4.934410e+05,4.934410e+05,4.934410e+05,4.934410e+05
mean,5.107197e+04,3.042671e+03,5.425967e+03,3.708233e+03,2.227427e+02,4.279809e+02,6.160078e+04,1.021980e+04
std,1.656046e+06,4.052455e+05,3.028736e+05,1.213474e+05,1.451766e+04,3.740105e+04,6.395431e+05,1.915077e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,5.730000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.900000e+08,2.164307e+08,1.420225e+08,5.086000e+07,5.003991e+06,1.154171e+07,3.283146e+07,5.000000e+07


,X1,X2,X3,X4,X5,X6,X7,X8
count,5.065130e+05,5.065130e+05,5.065130e+05,5.065130e+05,5.065130e+05,5.065130e+05,5.065130e+05,5.065130e+05
mean,4.965324e+04,3.594652e+03,6.016074e+03,4.000847e+03,2.039064e+02,3.460443e+02,6.273838e+04,1.066476e+04
std,1.647982e+06,5.701597e+05,2.185682e+05,8.197853e+04,1.156378e+04,3.068168e+04,6.455918e+05,1.870771e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,5.700000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.900000e+08,3.357371e+08,7.913093e+07,1.986000e+07,3.100000e+06,1.018211e+07,3.278319e+07,5.000000e+07


,X1,X2,X3,X4,X5,X6,X7,X8
count,5.215660e+05,5.215660e+05,5.215660e+05,5.215660e+05,5.215660e+05,5.215660e+05,5.215660e+05,5.215660e+05
mean,4.996368e+04,3.872203e+03,4.449279e+03,4.441739e+03,1.986803e+02,3.376526e+02,6.410221e+04,1.125153e+04
std,1.623167e+06,5.717576e+05,1.923967e+05,9.522327e+04,1.037671e+04,3.005550e+04,6.513732e+05,1.840652e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,5.750000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.900000e+08,3.346185e+08,7.913093e+07,3.000000e+07,2.900067e+06,1.020815e+07,3.273456e+07,5.000000e+07


,X1,X2,X3,X4,X5,X6,X7,X8
count,5.438230e+05,5.438230e+05,5.438230e+05,5.438230e+05,5.438230e+05,5.438230e+05,5.438230e+05,5.438230e+05
mean,4.767450e+04,3.674215e+03,5.989938e+03,4.462179e+03,2.572643e+02,4.353697e+02,6.395352e+04,1.230428e+04
std,1.590177e+06,5.750387e+05,2.448612e+05,1.169876e+05,1.835657e+04,3.916726e+04,6.508835e+05,2.063124e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,8.510000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.900000e+08,3.320671e+08,1.090412e+08,4.500000e+07,7.000000e+06,1.001083e+07,3.000000e+07,7.600000e+07
